In [ ]:
import warnings
warnings.filterwarnings('ignore')

from ai_council.council import *
from ai_council.prompts import *
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import PydanticOutputParser
from ai_council.vector import *

[(k["llm"] , k["name"]) for k in MODELS]

[(OllamaLLM(model='starling-lm'), 'starling-1m'),
 (OllamaLLM(model='ministral-3'), 'minstral-3'),
 (OllamaLLM(model='mistral:7b'), 'mistral:7b'),
 (OllamaLLM(model='phi3:mini'), 'phi3:mini'),
 (OllamaLLM(model='gemma2:9b'), 'gemma2:9b'),
 (OllamaLLM(model='gpt-oss:20b'), 'gpt-oss:20b')]

In [2]:
vs = get_vector_db()
input_prompt = "Summarize the document"
context = vs.similarity_search(input_prompt)
responses , user_prompt = generate_expert_response(input_prompt, context)
for response in responses:
    print(response)

Response from starling-1m
Response generated by starling-1m with id r_0
Response from minstral-3
Response generated by minstral-3 with id r_1
Response from mistral:7b
Response generated by mistral:7b with id r_2
Response from phi3:mini
Response generated by phi3:mini with id r_3
Response from gpt-oss:20b
Response generated by gpt-oss:20b with id r_5
{'response_id': 'r_0', 'model_id': 'expert_2', 'text': '1. Final answer: The document focuses on alignment of interest, governance, and transparency in private equity. It discusses reporting standards for quarterly and annual financial information, LPAC best practices, carry clawback considerations, and financial reporting.\n2. Brief reasoning: The context provided outlines key principles and provides appendices discussing various aspects related to private equity practices. The document emphasizes the importance of clear communication with investors through regular reports on fund performance and portfolio company information.\n3. Snippets

In [3]:
scoring_matrix = generate_scores(responses, user_prompt)

starling-1m
Could not parse r_0 by starling-1m due to expected string or bytes-like object, got 'NoneType'
Could not parse r_1 by starling-1m due to 'scores'
Could not parse r_2 by starling-1m due to expected string or bytes-like object, got 'NoneType'
Could not parse r_3 by starling-1m due to unterminated string literal (detected at line 2) (<unknown>, line 2)
Scoring complete for r_5 by starling-1m
minstral-3
Scoring complete for r_0 by minstral-3
Scoring complete for r_1 by minstral-3
Scoring complete for r_2 by minstral-3
Scoring complete for r_3 by minstral-3
Scoring complete for r_5 by minstral-3
mistral:7b
Scoring complete for r_0 by mistral:7b
Scoring complete for r_1 by mistral:7b
Scoring complete for r_2 by mistral:7b
Scoring complete for r_3 by mistral:7b
Scoring complete for r_5 by mistral:7b
phi3:mini
Scoring complete for r_0 by phi3:mini
Scoring complete for r_1 by phi3:mini
Scoring complete for r_2 by phi3:mini
Scoring complete for r_3 by phi3:mini
Scoring complete for r

In [4]:
audit , audit_prompt= generate_audit_report(input_prompt, responses, scoring_matrix)

input_variables=['output_format', 'responses', 'scoring_matrix', 'user_prompt'] input_types={} partial_variables={} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['output_format', 'responses', 'scoring_matrix', 'user_prompt'], input_types={}, partial_variables={}, template='\n    SYSTEM: You are an independent auditor whose job is to inspect a scoring matrix produced by peer models and detect bias, collusion, or anomalous scoring patterns. Return only the JSON described below.\n\n    USER: We provide:\n    1) original_prompt: {user_prompt}\n    2) responses: a JSON list of response objects:\n    {responses}\n    3) scoring_matrix: a JSON object where keys are scorer_ids and values are dictionaries mapping response_id -> score_obj\n    e.g., {scoring_matrix}\n\n    Task:\n    1) Inspect scoring patterns for the following anomalies:\n    - Self-scoring or allowed self-favoring (scorer giving systematically higher scores to a single partner)\n    - Collusion: 

In [5]:
print_with_bold(audit)

```json
{
  "audit_id": "example-audit-123",
  "flags": [
    {
      "scorer_id": "minstral-3",
      "issue": "Self-scoring",
      "severity": "High"
    }
  ],
  "drops": [],
  "explanation": "The scorer 'minstral-3' consistently gives significantly higher scores to response 'r_0' compared to other scorers. This suggests potential self-favoring or bias.",
  "normalization": {
    "minstral-3": 0.85
  }
}
``` 


Explanation:

* `audit_id`: A unique identifier for this audit run.
* `flags`: An array of objects, each detailing a detected anomaly. Here, we have one flag:
    * `"scorer_id"`: The ID of the scorer exhibiting the issue ("minstral-3").
    * `"issue"`:  A description of the anomaly ("Self-scoring").
    * `"severity"`: A level indicating the seriousness of the issue (e.g., "High", "Medium", "Low").
* `drops`: An array to list scorer IDs that are completely excluded from the aggregation due to severe issues.  This example doesn't use it as we only need normalization for one

In [6]:
scoring_matrix, averages_json, best_response  = audited_scoring_matrix(audit, scoring_matrix, responses)

In [7]:
# response_ids = [k['response_id'] for k in responses]
# audit_json = ast.literal_eval('{'+extract_first_curly_balanced(audit)+"}")
# for model in audit_json['normalization']:
#     for j in scoring_matrix[model]:
#         scoring_matrix[model][j]['total'] *= audit_json['normalization'][model]
# pretty_print_json(scoring_matrix)

In [8]:
# scoring_matrix

In [9]:
# responses[response_ids.index('r_1')]

In [10]:
# audit_json

In [11]:
# scoring_matrix[model][j]['total']

In [12]:
# import json

# # your_data = {...}  # put your full dictionary here

# def compute_average_totals(data):
#     totals = {}      # collects all totals per response_id
#     counts = {}      # counts how many values per response_id

#     for model_data in data.values():
#         for response_id, response_content in model_data.items():
#             total_value = response_content["total"]

#             if response_id not in totals:
#                 totals[response_id] = 0
#                 counts[response_id] = 0

#             totals[response_id] += total_value
#             counts[response_id] += 1

#     # compute averages
#     averages = {response_id: totals[response_id] / counts[response_id]
#                 for response_id in totals}

#     return averages


# # Example usage:
# averages_json = compute_average_totals(scoring_matrix)
# print(json.dumps(averages_json, indent=4))


In [13]:
# max(averages_json)

In [14]:
# print_with_bold(responses[response_ids.index(max(averages_json))]['text'])